# Empirical downscaling of climate data with DL

This notebook is modified based on the original DL4DS tutorial notebook to make it working at NCI. It implemented from the following paper, titled: **DL4DS - Deep Learning for empirical DownScaling** <br>
Link: https://arxiv.org/pdf/2205.08967.pdf <br>

In [ ]:
import numpy as np
import xarray as xr
import ecubevis as ecv
import dl4ds as dds
import scipy as sp
import netCDF4 as nc
import climetlab as cml
from numba import cuda 
device = cuda.get_current_device()
device.reset()  

In [ ]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import models

In [ ]:
# Change to your working directory
import os
os.chdir(YOUR_WORKING_DIRECTORY)

## MAELSTROM downscaling benchmark

**MAchinE Learning for Scalable meTeoROlogy and climate** (MAELSTROM) is a joint project by several climate research organizations that provides various machine learning datasets for climate research. 

In this notebook, the **MAELSTROM 2m temperature downscaling dataset** has been used. You can downlaod the dataset via cml.load_dataset() method. You can specify the cache directory via the cml.settings.set() method. 

Or, you can access a copy of dataset directly from Gadi local file system "/g/data/dk92/apps/dl4ds/data/maelstrom/2m_temperature_downsacaling_dataset". 

More details can be found in the following white paper: https://www.maelstrom-eurohpc.eu/content/docs/uploads/doc6.pdf

In [ ]:
#cml.settings.get("cache-directory") # Find the current cache directory
# Change the value of the setting
#cml.settings.set("cache-directory", "/big-disk/climetlab-cache")
# Python kernel restarted

In [ ]:
# Download MAELSTROM datasets
# # No need to run the following lines at NCI Gadi.
#cmlds_train = cml.load_dataset("maelstrom-downscaling", dataset="training")
#cmlds_val = cml.load_dataset("maelstrom-downscaling", dataset="validation")
#cmlds_test = cml.load_dataset("maelstrom-downscaling", dataset="testing")
#cmlds_train.to_xarray().to_netcdf("cmlds_train.nc")
#cmlds_val.to_xarray().to_netcdf("cmlds_val.nc")
#cmlds_test.to_xarray().to_netcdf("cmlds_test.nc")

### Load dataset From local file system

In [ ]:
cmlds_train = cml.load_source('file',"/g/data/dk92/apps/dl4ds/data/maelstrom/2m_temperature_downsacaling_dataset/cmlds_train.nc")
cmlds_val   = cml.load_source('file',"/g/data/dk92/apps/dl4ds/data/maelstrom/2m_temperature_downsacaling_dataset/cmlds_val.nc")
cmlds_test  = cml.load_source('file',"/g/data/dk92/apps/dl4ds/data/maelstrom/2m_temperature_downsacaling_dataset/cmlds_test.nc")

t2m_hr_train = cmlds_train.to_xarray().t2m_tar
t2m_hr_test = cmlds_test.to_xarray().t2m_tar
t2m_hr_val = cmlds_val.to_xarray().t2m_tar

t2m_lr_train = cmlds_train.to_xarray().t2m_in
t2m_lr_test = cmlds_test.to_xarray().t2m_in
t2m_lr_val = cmlds_val.to_xarray().t2m_in

z_hr_train = cmlds_train.to_xarray().z_tar
z_hr_test = cmlds_test.to_xarray().z_tar
z_hr_val = cmlds_val.to_xarray().z_tar

z_lr_train = cmlds_train.to_xarray().z_in
z_lr_test = cmlds_test.to_xarray().z_in
z_lr_val = cmlds_val.to_xarray().z_in

In [ ]:
z_hr_train

In [ ]:
print(t2m_hr_train.shape,t2m_hr_val.shape, t2m_hr_test.shape)
print(t2m_lr_train.shape,t2m_lr_val.shape, t2m_lr_test.shape)
print(z_hr_train.shape,z_hr_val.shape, z_hr_test.shape)
print(z_lr_train.shape,z_lr_val.shape, z_lr_test.shape)

In [ ]:
# The high resolution and low resolution datasets for training are shown below.
( ecv.plot(t2m_hr_train, interactive=True) + ecv.plot(t2m_lr_train, interactive=True) )

Here we take care of the scaling/normalization of values before training our networks. In this exmample, we center wrt the global mean and scale wrt the global standard deviation.


In [ ]:
t2m_scaler_train = dds.StandardScaler(axis=None)
# Calculate the mean and standard deviation of t2m_hr_train for later scaling.
t2m_scaler_train.fit(t2m_hr_train)  
# Perform standardization by centering and scaling.
y_train = t2m_scaler_train.transform(t2m_hr_train)
y_test = t2m_scaler_train.transform(t2m_hr_test)
y_val = t2m_scaler_train.transform(t2m_hr_val)

x_train = t2m_scaler_train.transform(t2m_lr_train)
x_test = t2m_scaler_train.transform(t2m_lr_test)
x_val = t2m_scaler_train.transform(t2m_lr_val)

z_scaler_train = dds.StandardScaler(axis=None)
z_scaler_train.fit(z_hr_train)  
y_z_train = z_scaler_train.transform(z_hr_train)
y_z_test = z_scaler_train.transform(z_hr_test)
y_z_val = z_scaler_train.transform(z_hr_val)

x_z_train = z_scaler_train.transform(z_lr_train)
x_z_test = z_scaler_train.transform(z_lr_test)
x_z_val = z_scaler_train.transform(z_lr_val)


In [ ]:
ecv.plot((x_train[0], y_train[0]), subplot_titles=('t2m coarsened', 't2m highres'))

In [ ]:
ecv.plot((x_z_train[0], y_z_train[0]), subplot_titles=('geopotential coarsened', 'geopotential highres'))

Add a unitary dimension called channel.

In [ ]:
y_train = y_train.expand_dims(dim='channel', axis=-1)
y_test = y_test.expand_dims(dim='channel', axis=-1)
y_val = y_val.expand_dims(dim='channel', axis=-1)

x_train = x_train.expand_dims(dim='channel', axis=-1)
x_test = x_test.expand_dims(dim='channel', axis=-1)
x_val = x_val.expand_dims(dim='channel', axis=-1)

y_z_train = y_z_train.expand_dims(dim='channel', axis=-1)
y_z_test = y_z_test.expand_dims(dim='channel', axis=-1)
y_z_val = y_z_val.expand_dims(dim='channel', axis=-1)

x_z_train = x_z_train.expand_dims(dim='channel', axis=-1)
x_z_test = x_z_test.expand_dims(dim='channel', axis=-1)
x_z_val = x_z_val.expand_dims(dim='channel', axis=-1)


The resulting shapes of the input arrays:

In [ ]:
print(y_train.shape, y_test.shape, y_val.shape)
print(x_train.shape, x_test.shape, x_val.shape)

print(x_z_train.shape, x_z_test.shape, x_z_val.shape)
print(y_z_train.shape, y_z_test.shape, y_z_val.shape)


## Train 

For training, DL4DS takes:

* HR data [mandatory, highres grid],
* LR data [optional, coarse grid],
* static variables [optional, highres grid],
* time-varyibng variables [optional, coarse to highres grid].

In this example, we do not use the LR data because it is just a coarsened version (via interpolation) of the HR t2m and z data. DL4DS carries out this interpolation on the fly by using the helping function dds.create_pair_hr_lr() (not to be called by the user). This process is done automatically inside the training loop (by calling one of the two Trainer classes in DL4DS), which we examplify here with the spc upsampling.

The function create_pair_hr_lr() create a pair of HR and LR square sub-patches. In this case, the LR corresponds to a coarsen version of the HR reference with land-ocean mask, topography and auxiliary predictors added as "image channels".

Parameters:
* array (np.ndarray): HR gridded data.
* array_lr (np.ndarray): LR gridded data. If not provided, then implicit/coarsened pairs are created from array.
* upsampling (str): String with the name of the upsampling method.
* scale (int): Scaling factor.
* patch_size (int or None): Size of the square patches to be extracted, in pixels for the HR grid.
* static_vars (None or list of 2D ndarrays, optional): Static variables such as elevation data or a binary land-ocean mask.
* predictors (np.ndarray, optional): Predictor variables in HR. To be concatenated to the LR version of array.
* season
* debug (bool, optional): If True, plots and debugging information are shown.
* interpolation (str, optional): Interpolation used when upsampling/downsampling the training samples. By default 'bicubic'.


In [ ]:
_ = dds.create_pair_hr_lr(y_train.values[0], 
                          None, 
                          'spc', 
                          8, 
                          None, 
                          None, 
                          y_z_train.values[0], 
                          None, 
                          True, 
                          interpolation='inter_area')

To train a model, one need the following data
- Must have the **high resolution data**
- Can provide the **low resolution data**. However, DL4DS creates the **low  resolution data** using build-in interpolation function, if none is provided. 

Below is an example of a call to the SupervisedTrainer class to run the training loop for 100 epochs. In this case, we train a network with a ResNet backbone, 8 residual blocks, in post-upsampling via subpixel convolution (implicit training pairs), and a localized convolutional block.

In [ ]:
ARCH_PARAMS = dict(n_filters=8,
                   n_blocks=8,
                   normalization=None,
                   dropout_rate=0.0,
                   dropout_variant='spatial',
                   attention=False,
                   activation='relu',
                   localcon_layer=True)

trainer = dds.SupervisedTrainer(
    backbone='resnet',
    upsampling='spc', 
    data_train=y_train, 
    data_val=y_val,
    data_test=y_test,
    data_train_lr=None, 
    data_val_lr=None,  
    data_test_lr=None, 
    scale=8,
    time_window=None, 
    static_vars=None,
    predictors_train=[y_z_train],
    predictors_val=[y_z_val],
    predictors_test=[y_z_test],
    interpolation='inter_area',
    patch_size=None, 
    batch_size=60, 
    loss='mae',
    epochs=100, 
    steps_per_epoch=None, 
    validation_steps=None, 
    test_steps=None, 
    learning_rate=(1e-3, 1e-4), lr_decay_after=1e4,
    early_stopping=False, patience=6, min_delta=0, 
    save=False, 
    save_path=None,
    show_plot=True, verbose=True, 
    device='CPU', 
    **ARCH_PARAMS)

trainer.run()


## Inference

Let's evaluate the results on holdout data -- the test split of the benchmark dataset that has not been used while training (updating the network weights).

In [ ]:
print (y_test.shape, y_z_test.shape)

In [ ]:

pred = dds.Predictor(
    trainer, 
    y_test, 
    scale=8, 
    array_in_hr=True,
    static_vars=None, 
    predictors=[y_z_test], 
    time_window=None,
    interpolation='inter_area', 
    batch_size=8,
    scaler=t2m_scaler_train,
    save_path=None,
    save_fname=None,
    return_lr=True,
    device='GPU')

unscaled_y_pred, coarsened_array = pred.run()


## Visualization

Below is the coarsened version of the holdout, passed to the trained model for inference:

In [ ]:
ind = 100
ecv.plot((coarsened_array[ind][:,:,0], coarsened_array[ind][:,:,1]))

Finally, let's see compare the groundtruth HR t2m and the downscaled t2m obtained with DL4DS for a single time step.

In [ ]:
unscaled_y_test = t2m_scaler_train.inverse_transform(y_test)

In [ ]:
ecv.plot((unscaled_y_test[ind].values,unscaled_y_pred[ind]), subplot_titles=('groundtruth t2m','downscaled t2m'))

We can also compare the groundtruth HR t2m and the downscaled t2m obtained with DL4DS in a synchronized way.

In [ ]:
( ecv.plot(unscaled_y_test.values) + ecv.plot(unscaled_y_pred, interactive=True) )